In [5]:
import re
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup

# Анализ данных сайта [ученые-исследователи.рф](https://xn----8sbfhdabdwf1afqu5baxe0f2d.xn--p1ai/)

Согласно описанному в [Приказе Минобрнауки РФ от 05.08.2021 N 715](https://normativ.kontur.ru/document?moduleId=1&documentId=400674) (действует с 01.03.2022) довольно большой перечень научных работников университетов должен проходить через конкурс, публикуемый на сайте ученые-исследователи.рф. Мне подсказали, что сайт вполне парсится, а самое главное поделились уже спарсенными данными о университетах, являющихся участниками [программы «Приоритет 2030»](https://priority2030.ru/).

## Парсим [ученые-исследователи.рф](https://xn----8sbfhdabdwf1afqu5baxe0f2d.xn--p1ai/)

для начала, с помощью любого поисковика ([пример](https://yandex.ru/search/?text=site%3axn----8sbfhdabdwf1afqu5baxe0f2d.xn--p1ai+%22%d0%9a%d1%80%d1%8b%d0%bc%d1%81%d0%ba%d0%b8%d0%b9+%d1%84%d0%b5%d0%b4%d0%b5%d1%80%d0%b0%d0%bb%d1%8c%d0%bd%d1%8b%d0%b9+%d1%83%d0%bd%d0%b8%d0%b2%d0%b5%d1%80%d1%81%d0%b8%d1%82%d0%b5%d1%82+%d0%b8%d0%bc%d0%b5%d0%bd%d0%b8+%d0%92.%d0%98.+%d0%92%d0%b5%d1%80%d0%bd%d0%b0%d0%b4%d1%81%d0%ba%d0%be%d0%b3%d0%be%22&lr=2) для яндекса), потребовалось собрать страницы университетов на анализируемом сайте. пример для «Крымский федеральный университет имени В.И. Вернадского» `site:xn----8sbfhdabdwf1afqu5baxe0f2d.xn--p1ai "Крымский федеральный университет имени В.И. Вернадского"`.

Получился [вот такой список](https://github.com/psalru/analytics/blob/9cbccf3f66be935fd7901193b46b6aba0ffde4a7/scientists_researchers_rf/data/university_list.csv), с привязкой к id университетов (у 25 университетов такие страницы найти не смог):

In [4]:
ul = pd.read_csv('data/university_list.csv')
ul.head()

,id,url
0,u1dpt3iyms,https://xn----8sbfhdabdwf1afqu5baxe0f2d.xn--p1...
1,8zd4kctsoz,NaN
2,ctiqcaywqm,https://xn----8sbfhdabdwf1afqu5baxe0f2d.xn--p1...
3,sisytkom7x,https://xn----8sbfhdabdwf1afqu5baxe0f2d.xn--p1...
4,chokaxj6xy,https://xn----8sbfhdabdwf1afqu5baxe0f2d.xn--p1...


Далее опишем рекурсивную функцию, которая позволит собрать нам данные по вакансиям конкретного университета, опубликованных на изучаемом сайте:

In [7]:
def get_vac_list(url, page=1, delay=1):
    org_url = '{0}?page={1}'.format(url, page)
    result = []

    time.sleep(delay)
    resp = requests.get(org_url)

    if resp.status_code == 200:
        soup = BeautifulSoup(resp.text, features='lxml')

        for tr in soup.select('table.tbl-vac > tbody > tr'):
            vac_id = tr.select_one('td:nth-child(2)').text.strip()
            pub_date = tr.select_one('td:nth-child(4)').text.strip()
            status = tr.select_one('td:nth-child(5)').text.strip()
            href = tr.select_one('td:nth-child(6) a')['href']

            result.append({
                'vac_id': vac_id,
                'pub_date': pub_date,
                'status': status,
                'url': href
            })

        if page == 1:
            pager_links = soup.select('div.pager > ul > li > a')

            if len(pager_links) > 0:
                last_pager_link_href = pager_links[-1]['href']
                count_of_pages = int(re.findall(r"\d*$", last_pager_link_href)[0])

                for i in range(2, count_of_pages + 1):
                    result.extend(get_vac_list(url, page=i))

    return result


vac_list = pd.DataFrame(get_vac_list(ul.loc[0, 'url']))
vac_list.head()

,vac_id,pub_date,status,url
0,VAC 67337,"27.08.2020, 10:13",Отменена,/public/vacancies/view/67337
1,VAC 67336,"27.08.2020, 10:04",Отменена,/public/vacancies/view/67336
2,VAC 67189,"26.08.2020, 13:57",Отменена,/public/vacancies/view/67189
3,VAC 67184,"26.08.2020, 13:45",Отменена,/public/vacancies/view/67184
4,VAC 67177,"26.08.2020, 13:24",Отменена,/public/vacancies/view/67177


Аккуратно собрав данные по вакансиям, можно пойти уже на страницы самих вакансий и забрать уже весь массив данных, необходимый нам для анализа:

In [10]:
def get_vac_data(url, delay=1):
    vac_url = url
    result = {'vac_url': vac_url}

    time.sleep(delay)
    resp = requests.get(vac_url)

    if resp.status_code == 200:
        soup = BeautifulSoup(resp.text, features='lxml')
        header_info_cells = soup.select('.scroll-frame__header-info .scroll-frame__header-info-col')

        for cell in header_info_cells:
            cell_data = re.sub(r'\n', '', cell.text)
            cell_data = re.sub(r'<(^>|.)*>', '', cell_data)
            cell_data = re.sub(r'\s+', ' ', cell_data)
            cell_data = cell_data.strip()
            colon_position = cell_data.find(':')
            key = cell_data[0:colon_position].strip().lower()
            value = cell_data[colon_position + 1:].strip().lower()

            result[key] = value

        data_rows = soup.select('.shadow-block')

        for row in data_rows:
            key_tag = row.select_one('.shadow-block_table-cell_label')

            if key_tag:
                key = re.sub(r':$', '', key_tag.text.strip().lower())
                value_tag = row.select_one('.shadow-block_table-cell:nth-child(2)')
                value = re.sub('\s+', ' ', value_tag.text.strip()) if value_tag else ''

                result[key] = value

    return result


vac_list['url'] = vac_list['url'].apply(lambda x: 'https://xn----8sbfhdabdwf1afqu5baxe0f2d.xn--p1ai{0}'.format(x))
vac_data = pd.Series(get_vac_data(vac_list.loc[0, 'url']))
vac_data

vac_url                                                                         https://xn----8sbfhdabdwf1afqu5baxe0f2d.xn--p1...
статус                                                                                                                   отменена
начало приема заявок                                                                                             01.10.2020 10:00
окончание приема заявок                                                                                          30.10.2020 17:00
дата проведения конкурса                                                                                         02.11.2020 10:00
организация                                                                     Федеральное государственное бюджетное образова...
должность                                                                       Лаборант центра адыговедения и адыгейской фило...
отрасль науки                                                                             

Далее аккуратно помучив несколько часов сайт [ученые-исследователи.рф](https://xn----8sbfhdabdwf1afqu5baxe0f2d.xn--p1ai/), был собран следующий массив данных:

In [16]:
vacancies = pd.read_csv('data/university_vacancies.csv', sep=';')
vacancies.head()

,id,short_name,org_url,vac_id,vac_pub_date,e-mail,vac_url,вакансия для выпускников вузов,годовое премирование,деятельность,...,трудовая деятельность,трудовые функции,условия премирования,ученая степень и звание,"фамилия, имя, отчество",дата проведения конкурса,начало приема заявок,окончание приема заявок,протокол решения комиссии,решение комиссии
0,qfosc1klbk,СКФУ,https://xn----8sbfhdabdwf1afqu5baxe0f2d.xn--p1...,VAC 82606,2021-09-08 14:44:00,ngandraburova@ncfu.ru,https://xn----8sbfhdabdwf1afqu5baxe0f2d.xn--p1...,Нет,0,Проведение исследования,...,Обобщать научные и (или) научно-технические ре...,Обобщение научных (научно-технических) результ...,NaN,кандидат физико-математических наук,Масалов Сергей Валерьевич,NaN,NaN,NaN,NaN,NaN
1,qfosc1klbk,СКФУ,https://xn----8sbfhdabdwf1afqu5baxe0f2d.xn--p1...,VAC 81256,2021-08-06 18:00:00,ngandraburova@ncfu.ru,https://xn----8sbfhdabdwf1afqu5baxe0f2d.xn--p1...,Нет,0,Доведение до всеобщего сведения научных (научн...,...,Анализировать научную и (или) научно-техническ...,Обобщение научных (научно-технических) результ...,NaN,кандидат юридических наук,Масалов Сергей Валерьевич,NaN,NaN,NaN,NaN,NaN
2,qfosc1klbk,СКФУ,https://xn----8sbfhdabdwf1afqu5baxe0f2d.xn--p1...,VAC 81248,2021-08-06 17:22:00,ngandraburova@ncfu.ru,https://xn----8sbfhdabdwf1afqu5baxe0f2d.xn--p1...,Нет,0,Формирование научного коллектива,...,"Проводить исследования, эксперименты, наблюден...",Постановка задач исследования научному коллект...,NaN,доцент кандидат наук,Масалов Сергей Валерьевич,2021-08-27 09:00:00,2021-08-06 18:00:00,2021-08-26 18:00:00,NaN,NaN
3,qfosc1klbk,СКФУ,https://xn----8sbfhdabdwf1afqu5baxe0f2d.xn--p1...,VAC 70745,2020-10-30 18:48:00,smasalov@ncfu.ru,https://xn----8sbfhdabdwf1afqu5baxe0f2d.xn--p1...,Нет,0,ИНОЕ,...,Координировать деятельность научных коллективо...,"ИНОЕ - организует выполнение фундаментальных, ...",NaN,кандидат наук доктор наук,Масалов Сергей Валерьевич,2020-11-20 09:00:00,2020-10-30 19:00:00,2020-11-19 19:00:00,NaN,NaN
4,qfosc1klbk,СКФУ,https://xn----8sbfhdabdwf1afqu5baxe0f2d.xn--p1...,VAC 64558,2020-07-24 16:51:00,smasalov@ncfu.ru,https://xn----8sbfhdabdwf1afqu5baxe0f2d.xn--p1...,Нет,0,ИНОЕ,...,Анализировать научную и (или) научно-техническ...,Постановка задач исследования научному коллект...,NaN,кандидат наук,Масалов Сергей Валерьевич,2020-08-25 09:00:00,2020-07-31 15:00:00,2020-08-20 15:00:00,NaN,NaN


На этом **всё**, т.к. цель данного документа была приблизительно показать механики, которые позволят Вам собрать данные с сайта [ученые-исследователи.рф](https://xn----8sbfhdabdwf1afqu5baxe0f2d.xn--p1ai/). Сам анализ предлагаю продолжить в соответсвующем [dashboard-е на DataLens](https://datalens.yandex/hr3o937ifwi29).

Если у Вас есть вопросы, а ещё лучше улучшения к данному материалу, приглашаю Вас к открытому обсуждению в Телеграм [группе](https://t.me/psalgroup) посвящённой «Приоритету 2030» или ещё лучше, **присылайте свои issues-ы** сразу [в репозиторий](https://github.com/psalru/analytics/issues).